# Import Dependencies 

In [8]:
import os 
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Test Environment 

In [9]:
env = gym.make('CarRacing-v2', render_mode='human')

In [10]:
env.action_space

Box([-1.  0.  0.], 1.0, (3,), float32)

In [11]:
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [12]:
for ep in range(1, 3):
    env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward
    print(f"Episode: {ep} Score: {score}")
env.close()

Episode: 1 Score: -27.536231884058353
Episode: 2 Score: -29.328621908127605


# Train Model 

In [14]:
train_env = DummyVecEnv([lambda: gym.make('CarRacing-v2', render_mode='human')])

log_path = os.path.join('Training', 'Logs')
model = PPO('CnnPolicy', train_env, verbose=1, tensorboard_log=log_path)

model.learn(total_timesteps=100000)


/home/zero/Downloads/RL/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env in a VecTransposeImage.
Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 48   |
|    iterations      | 1    |
|    time_elapsed    | 42   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 39           |
|    iterations           | 2            |
|    time_elapsed         | 102          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0050105834 |
|    clip_fraction        | 0.0447       |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.26        |
|    explained_variance   | -0.00595     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.169        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00559     |
|    std     

# Save Model 

In [15]:
ppo_path = os.path.join('Training', 'Saved_Models', 'PPO_Driving_Model')
model.save(ppo_path)

In [16]:
del model 
model = PPO.load(ppo_path, train_env)

Wrapping the env in a VecTransposeImage.


# Evaluate and Test Model

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

In [ ]:
for ep in range(1, 3):
    obs, _ = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward
    print(f"Episode: {ep} Score: {score}")
env.close()

# Viewing Tensorboard Logs

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [ ]:
!tensorboard --logdir={training_log_path}